# DATA CLEANING

In [2]:
import pandas as pd
import numpy as np

data_set = pd.read_csv('airbnb_lisbon_1480_2017-07-27.csv')
data_set.head()

data_set2 = data_set.apply(lambda x: x.astype(str).str.lower()) #this converts dataframe to lowercase
col_to_drop = ["country", "borough", "room_id", "survey_id", "host_id", "bathrooms", "minstay", "last_modified","latitude","longitude","location"]
my_data = data_set2.drop(col_to_drop, axis=1)
my_data[['reviews','overall_satisfaction','price']] = my_data[['reviews','overall_satisfaction','price']].astype('float')
#my_data.dtypes


#dropped_col_data = dropped_col_data[dropped_col_data.reviews !=0]
new_data = my_data[my_data["reviews"] != 0]

new_data.tail(20)
new_data.to_csv('cleaned_data.csv', index=False)

# IMPORT CLEAN DATA

In [64]:
import pandas as pd
import numpy as np

clean_data = pd.read_csv('cleaned_data.csv')
print(clean_data.head())

print(clean_data.mean(axis=0))
print(clean_data.max(axis=0))
print(clean_data.std(axis=0))
print(clean_data.median(axis=0))

     room_type    city       neighborhood  reviews  overall_satisfaction  \
0  shared room  lisbon      santo antónio     19.0                   4.5   
1  shared room  lisbon     avenidas novas      4.0                   3.5   
2  shared room  lisbon  santa maria maior     38.0                   4.5   
3  shared room  lisbon     avenidas novas      9.0                   4.0   
4  shared room  lisbon            arroios      1.0                   0.0   

   accommodates  bedrooms  price                                         name  
0             4       1.0   30.0   low cost hostel marques gardens 4 bed dorm  
1             6       1.0   39.0                        room in lisbon center  
2             8       1.0   39.0  bed in a 8-bed dorm @ this is lisbon hostel  
3             6       1.0   37.0                  shared mixed bunkbed room 1  
4             2       1.0   96.0                              nice room anjos  
reviews                 35.814159
overall_satisfaction     3.93

In [65]:
clean_data = clean_data[clean_data["reviews"] > 35]
print(clean_data.mean(axis=0))
print(clean_data.std(axis=0))
#clean_data.tail()

reviews                 85.260086
overall_satisfaction     4.664067
accommodates             4.014524
bedrooms                 1.505648
price                   72.728080
dtype: float64
reviews                 49.287568
overall_satisfaction     0.289219
accommodates             2.226918
bedrooms                 0.970839
price                   41.342061
dtype: float64


In [ ]:
# clean_data = clean_data[clean_data["reviews"]]

# DATA FOR RATINGS

In [66]:
drop_list = ["city", "accommodates", "name", "room_type", "bedrooms", "price"]
ratings = clean_data.drop(drop_list, axis = 1)

ratings['Normalized Review Quantity'] = (((ratings['reviews']-min(ratings['reviews']))/(135-min(ratings['reviews'])))*5)
ratings['Normalized Review Quantity'] = np.where(ratings['reviews'] > 135, 5, ratings['Normalized Review Quantity'])
#print(ratings.sort_values(["Normalized Review Quantity"],ascending=False))

ratings['Normalized Satisfaction'] = (((ratings['overall_satisfaction']-min(ratings['overall_satisfaction']))/(4.7-min(ratings['overall_satisfaction'])))*5)
ratings['Normalized Satisfaction'] = np.where(ratings['overall_satisfaction'] > 4.7, 5, ratings['Normalized Satisfaction'])
ratings['Normalized Satisfaction'] = np.where(ratings['overall_satisfaction'] < 4.3, 0, ratings['Normalized Satisfaction'])
print(ratings.sort_values(["Normalized Satisfaction"],ascending=False))

ratings['ratings_weight'] = (ratings['Normalized Review Quantity']*.5) + (ratings['Normalized Satisfaction']*.5)
ratings.sort_values(['Normalized Review Quantity'], ascending = False)

ratings_agg = round((ratings.groupby(['neighborhood']).agg(['mean'])),2)
ratings_agg["reviews"] = ratings_agg['reviews'].astype('int')
ratings_agg = ratings_agg.reset_index(level = 0)
ratings_agg.columns = ["Neighborhood", "AVG Reviews / Listing", "AVG User Rating","Normalized Review Quantity", "Normalized Satisfaction", "Weighted Ratings"]
ratings_agg["Rank"] = ratings_agg['Weighted Ratings'].rank(ascending = False).astype('int')
ratings_agg = ratings_agg[['Rank',"Neighborhood","Weighted Ratings", "Normalized Satisfaction", "Normalized Review Quantity", "AVG User Rating", "AVG Reviews / Listing",]]
#ratings_agg.head(50)
ratings_agg.sort_values(["Rank"],ascending = True)

# ratings_mean.sort_values(['overall_satisfaction'], ascending=False)

            neighborhood  reviews  overall_satisfaction  \
5948        misericórdia    191.0                   5.0   
4650       santo antónio     67.0                   5.0   
4648       santo antónio     83.0                   5.0   
4645       santo antónio     44.0                   5.0   
4634       santo antónio     75.0                   5.0   
4631   santa maria maior     56.0                   5.0   
4627       santo antónio     75.0                   5.0   
4622       santo antónio     38.0                   5.0   
4611       santo antónio     84.0                   5.0   
4608       santo antónio     62.0                   5.0   
8415             arroios     36.0                   5.0   
4594        misericórdia     41.0                   5.0   
4586       santo antónio     40.0                   5.0   
4585         são vicente    157.0                   5.0   
4583   santa maria maior     64.0                   5.0   
8429       santo antónio     38.0                   5.0 

,Rank,Neighborhood,Weighted Ratings,Normalized Satisfaction,Normalized Review Quantity,AVG User Rating,AVG Reviews / Listing
15,1,olivais,3.54,4.39,2.69,4.64,116
18,2,santa maria maior,3.37,4.31,2.44,4.67,92
14,3,misericórdia,3.25,4.23,2.28,4.66,89
1,4,alcântara,3.17,4.23,2.11,4.61,79
19,5,santo antónio,3.13,4.26,2.00,4.69,79
11,6,estrela,3.12,4.28,1.96,4.66,79
2,7,alvalade,3.11,4.38,1.85,4.68,77
4,7,arroios,3.11,4.28,1.95,4.67,77
21,9,são vicente,3.07,4.12,2.02,4.62,81
5,10,avenidas novas,3.05,4.51,1.58,4.74,70


# DATA FOR PRICE

In [48]:
drop_list = ["city", "accommodates", "name", "reviews", "overall_satisfaction","room_type"]
price = clean_data.drop(drop_list, axis = 1)
price['bedrooms'] = price['bedrooms'].map(lambda x: 1 if x==0 else x)
price['price/br'] = (price['price']*.9)/price["bedrooms"]
drop_price = ["price","bedrooms"]
price = price.drop(drop_price, axis = 1)
price_agg = round(price.groupby(['neighborhood']).agg(['mean']),2)
price_agg = price_agg.reset_index(level = 0)
price_agg.columns = ["Neighborhood","Price / BR (€)"]
price_agg["Rank"] = price_agg['Price / BR (€)'].rank(ascending = True).astype('int')
price_agg = price_agg[["Rank","Neighborhood","Price / BR (€)"]]
price_agg.sort_values('Price / BR (€)',ascending=False)



#price.groupby(['neighborhood']).agg(['mean'])
#price.head(100)

,Rank,Neighborhood,Price / BR (€)
18,22,santa maria maior,49.24
5,21,avenidas novas,48.39
14,20,misericórdia,47.32
16,19,parque das nações,47.30
11,18,estrela,45.33
19,17,santo antónio,45.16
9,16,campo de ourique,44.80
21,15,são vicente,43.30
20,14,são domingos de benfica,40.46
4,13,arroios,37.17


# MERGE RATINGS & PRICE

In [49]:
ratings_price_agg = ratings_agg.merge(price_agg, on = 'Neighborhood')
ratings_price_agg.columns = ["Ratings Rank", "Neighborhood","Weighted Ratings", "AVG User Rating", "AVG Reviews / Listing", "Normalized Satisfaction", "Normalized Review Quantity","Price Rank","Price / BR (€)"]
ratings_price_agg = ratings_price_agg[["Ratings Rank", "Price Rank", "Neighborhood","Weighted Ratings", "Price / BR (€)", "AVG User Rating", "AVG Reviews / Listing", "Normalized Satisfaction", "Normalized Review Quantity"]]
ratings_price_agg.sort_values("Ratings Rank")

,Ratings Rank,Price Rank,Neighborhood,Weighted Ratings,Price / BR (€),AVG User Rating,AVG Reviews / Listing,Normalized Satisfaction,Normalized Review Quantity
15,1,2,olivais,3.54,28.90,4.64,116,4.39,2.69
18,2,22,santa maria maior,3.37,49.24,4.67,92,4.31,2.44
14,3,20,misericórdia,3.25,47.32,4.66,89,4.23,2.28
1,4,11,alcântara,3.17,36.42,4.61,79,4.23,2.11
19,5,17,santo antónio,3.13,45.16,4.69,79,4.26,2.00
11,6,18,estrela,3.12,45.33,4.66,79,4.28,1.96
2,7,8,alvalade,3.11,34.66,4.68,77,4.38,1.85
4,7,13,arroios,3.11,37.17,4.67,77,4.28,1.95
21,9,15,são vicente,3.07,43.30,4.62,81,4.12,2.02
5,10,21,avenidas novas,3.05,48.39,4.74,70,4.51,1.58


In [44]:
ratings_price_agg["Weighted Ratings"].corr(ratings_price_agg["Price / BR (€)"])

0.41851878198132364

# --------------------------------------------------------------------------------------------------

# BAG OF WORDS

In [6]:
clean_data.head()

,room_type,city,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,name
0,shared room,lisbon,santo antónio,19.0,4.5,4,1.0,30.0,low cost hostel marques gardens 4 bed dorm
1,shared room,lisbon,avenidas novas,4.0,3.5,6,1.0,39.0,room in lisbon center
2,shared room,lisbon,santa maria maior,38.0,4.5,8,1.0,39.0,bed in a 8-bed dorm @ this is lisbon hostel
3,shared room,lisbon,avenidas novas,9.0,4.0,6,1.0,37.0,shared mixed bunkbed room 1
4,shared room,lisbon,arroios,1.0,0.0,2,1.0,96.0,nice room anjos


In [60]:
drop_list = ["city", "accommodates", "room_type", "bedrooms", "price"]
bow_ratings = clean_data.drop(drop_list, axis = 1)

bow_ratings['Normalized Review Quantity'] = (((bow_ratings['reviews']-min(bow_ratings['reviews']))/(max(bow_ratings['reviews'])-min(bow_ratings['reviews'])))*5)
bow_ratings['ratings_weight'] = (bow_ratings['Normalized Review Quantity']*.5) + (bow_ratings['overall_satisfaction']*.5)
bow_ratings.sort_values(['ratings_weight'], ascending = False)

#bow_ratings['ratings_weight'] = (ratings['reviews']*(0.4)) + (ratings['overall_satisfaction']*(0.6))
#bow_ratings.sort_values('ratings_weight', ascending = False).head()

# Limit to weighting over 100
ratings_100 = bow_ratings[(bow_ratings["ratings_weight"] > 2) & (bow_ratings['neighborhood'] == 'olivais')]
ratings_100.head(200)

,neighborhood,reviews,overall_satisfaction,name,Normalized Review Quantity,ratings_weight
173,olivais,438.0,4.5,open space in lisbon close airport,4.878641,4.689320
187,olivais,116.0,4.5,lisboa junto à expo/gare do oriente,0.970874,2.735437
3548,olivais,145.0,4.5,beautiful apt in expo area,1.322816,2.911408
6161,olivais,71.0,4.5,"encarnação - lisboa, 2 quartos",0.424757,2.462379
8205,olivais,68.0,4.5,"lisboa,1bd near aeroporto/p.nações",0.388350,2.444175
10813,olivais,47.0,4.5,cosy apartment near lisbon airport,0.133495,2.316748
10816,olivais,160.0,5.0,lisbon - bedroom + breakfast,1.504854,3.252427
10900,olivais,175.0,5.0,room with bathroom in lisbon,1.686893,3.343447
10903,olivais,48.0,4.5,private third floor + kitchen and living room,0.145631,2.322816
10932,olivais,114.0,4.5,lisbon private double room (2km airport),0.946602,2.723301


In [61]:
corpus = ratings_100["name"].values
corpus


array(['open space in lisbon close airport',
       'lisboa junto à expo/gare do oriente',
       'beautiful apt in expo area', 'encarnação - lisboa, 2 quartos ',
       'lisboa,1bd near aeroporto/p.nações',
       'cosy apartment near lisbon airport',
       'lisbon - bedroom + breakfast', 'room with bathroom in lisbon',
       'private third floor + kitchen and living room',
       'lisbon private double room (2km airport)',
       'lisbon private room two beds near airport',
       'private room + kitchen + living room',
       'próximo do aeroporto/ next to the airport',
       'luminoso espaço e verde em redor',
       'private room + kitchen and living room',
       'suite+privat bathr.-amazing lisb view-near airport',
       'cozy room close to airport/meo arena',
       'room near aiport (2km) with subway/bus',
       'private room near the airport + kitchen use',
       'pleasant and much sympathy',
       'room with private bathroom near airport',
       'quiet rooms close to

In [62]:
import sklearn.feature_extraction.text
from sklearn.feature_extraction.text import CountVectorizer

stop_words = ['all', 'six', 'less', 'being', 'indeed', 'over', 'move', 'anyway', 'fifty', 'four', 'not', 'own', 'through', 'yourselves', 'go', 'where', 'mill', 'only', 'find', 'before', 'one', 'whose', 'system', 'how', 'somewhere', 'with', 'thick', 'show', 'had', 'enough', 'should', 'to', 'must', 'whom', 'seeming', 'under', 'ours', 'has', 'might', 'thereafter', 'latterly', 'do', 'them', 'his', 'around', 'than', 'get', 'very', 'de', 'none', 'cannot', 'every', 'whether', 'they', 'front', 'during', 'thus', 'now', 'him', 'nor', 'name', 'several', 'hereafter', 'always', 'who', 'cry', 'whither', 'this', 'someone', 'either', 'each', 'become', 'thereupon', 'sometime', 'side', 'two', 'therein', 'twelve', 'because', 'often', 'ten', 'our', 'eg', 'some', 'back', 'up', 'namely', 'towards', 'are', 'further', 'beyond', 'ourselves', 'yet', 'out', 'even', 'will', 'what', 'still', 'for', 'bottom', 'mine', 'since', 'please', 'forty', 'per', 'its', 'everything', 'behind', 'un', 'above', 'between', 'it', 'neither', 'seemed', 'ever', 'across', 'she', 'somehow', 'be', 'we', 'full', 'never', 'sixty', 'however', 'here', 'otherwise', 'were', 'whereupon', 'nowhere', 'although', 'found', 'alone', 're', 'along', 'fifteen', 'by', 'both', 'about', 'last', 'would', 'anything', 'via', 'many', 'could', 'thence', 'put', 'against', 'keep', 'etc', 'amount', 'became', 'ltd', 'hence', 'onto', 'or', 'con', 'among', 'already', 'co', 'afterwards', 'formerly', 'within', 'seems', 'into', 'others', 'while', 'whatever', 'except', 'down', 'hers', 'everyone', 'done', 'least', 'another', 'whoever', 'moreover', 'couldnt', 'throughout', 'anyhow', 'yourself', 'three', 'from', 'her', 'few', 'together', 'top', 'there', 'due', 'been', 'next', 'anyone', 'eleven', 'much', 'call', 'therefore', 'interest', 'then', 'thru', 'themselves', 'hundred', 'was', 'sincere', 'empty', 'more', 'himself', 'elsewhere', 'mostly', 'on', 'fire', 'am', 'becoming', 'hereby', 'amongst', 'else', 'part', 'everywhere', 'too', 'herself', 'former', 'those', 'he', 'me', 'myself', 'made', 'twenty', 'these', 'bill', 'cant', 'us', 'until', 'besides', 'nevertheless', 'below', 'anywhere', 'nine', 'can', 'of', 'your', 'toward', 'my', 'something', 'and', 'whereafter', 'whenever', 'give', 'almost', 'wherever', 'is', 'describe', 'beforehand', 'herein', 'an', 'as', 'itself', 'at', 'have', 'in', 'seem', 'whence', 'ie', 'any', 'fill', 'again', 'hasnt', 'inc', 'thereby', 'thin', 'no', 'perhaps', 'latter', 'meanwhile', 'when', 'detail', 'same', 'wherein', 'beside', 'also', 'that', 'other', 'take', 'which', 'becomes', 'you', 'if', 'nobody', 'see', 'though', 'may', 'after', 'upon', 'most', 'hereupon', 'eight', 'but', 'serious', 'nothing', 'such', 'why', 'a', 'off', 'whereby', 'third', 'i', 'whole', 'noone', 'sometimes', 'well', 'amoungst', 'yours', 'their', 'rather', 'without', 'so', 'five', 'the', 'first', 'whereas', 'once']

vectorizer = CountVectorizer(stop_words=stop_words)

x = vectorizer.fit_transform(corpus)

bow = vectorizer.vocabulary_
print(bow)

{'open': 37, 'space': 48, 'lisbon': 31, 'close': 17, 'airport': 4, 'lisboa': 30, 'junto': 27, 'expo': 24, 'gare': 26, 'oriente': 38, 'beautiful': 12, 'apt': 7, 'area': 8, 'encarnação': 22, 'quartos': 43, '1bd': 0, 'near': 36, 'aeroporto': 2, 'nações': 35, 'cosy': 18, 'apartment': 6, 'bedroom': 13, 'breakfast': 15, 'room': 46, 'bathroom': 11, 'private': 41, 'floor': 25, 'kitchen': 28, 'living': 32, 'double': 20, '2km': 1, 'beds': 14, 'próximo': 42, 'luminoso': 33, 'espaço': 23, 'verde': 53, 'em': 21, 'redor': 45, 'suite': 50, 'privat': 40, 'bathr': 10, 'amazing': 5, 'lisb': 29, 'view': 54, 'cozy': 19, 'meo': 34, 'arena': 9, 'aiport': 3, 'subway': 49, 'bus': 16, 'use': 52, 'pleasant': 39, 'sympathy': 51, 'quiet': 44, 'rooms': 47}


In [37]:
term_freq = x.toarray()
print(term_freq)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [63]:
bow_100 = pd.DataFrame(list(bow.items()), columns = ['Word',"Count"])
bow_100["Rank"] = bow_100['Count'].rank(ascending=False).astype('int')
bow_100 = bow_100[["Rank","Word","Count"]]
bow_100.sort_values("Count", ascending=False)

#bow_100.sort_values("Count", ascending=False)

,Rank,Word,Count
43,1,view,54
35,2,verde,53
50,3,use,52
52,4,sympathy,51
38,5,suite,50
48,6,subway,49
1,7,space,48
54,8,rooms,47
23,9,room,46
37,10,redor,45


# BAG OF WORDS - REFERENCE

In [ ]:
corpus = np.array(())
corpus = (np.array(np.concatenate([open(x, "r").readlines() for x in name_agg]))).tolist()
print(corpus)
vectorizer = CountVectorizer()

x = vectorizer.fit_transform(corpus)

bow = vectorizer.vocabulary_
print(bow)

# bag_of_words = vectorizer.get_feature_names()
# print(bag_of_words)

term_freq = x.toarray()
print(term_freq)


#corpus = sklearn.feature_extraction.text.CountVectorizer(docs, stop_words = stop_words)
#print(corpus)

# --------------------------------------------------------------------------------------------------

# DATA FOR ROOMTYPE

In [34]:
drop_list = ["city", "accommodates", "name", "reviews", "overall_satisfaction"]
price = clean_data.drop(drop_list, axis = 1)
price.head()

,room_type,neighborhood,bedrooms,price
0,shared room,santo antónio,1.0,30.0
1,shared room,avenidas novas,1.0,39.0
2,shared room,santa maria maior,1.0,39.0
3,shared room,avenidas novas,1.0,37.0
4,shared room,arroios,1.0,96.0


In [36]:
price['price/br'] = price['price']/price["bedrooms"]
price.head(20)

,room_type,neighborhood,bedrooms,price,price/br
0,shared room,santo antónio,1.0,30.0,30.0
1,shared room,avenidas novas,1.0,39.0,39.0
2,shared room,santa maria maior,1.0,39.0,39.0
3,shared room,avenidas novas,1.0,37.0,37.0
4,shared room,arroios,1.0,96.0,96.0
5,shared room,campo de ourique,1.0,90.0,90.0
6,shared room,lumiar,1.0,96.0,96.0
7,shared room,parque das nações,1.0,84.0,84.0
8,shared room,avenidas novas,1.0,66.0,66.0
9,shared room,areeiro,1.0,58.0,58.0


# SWITCH ROOM TYPE TO VARIABLE

In [39]:
drop_price = ["price"]
price2 = price.drop(drop_price, axis = 1)
price3 = price2.pivot_table(index = ["neighborhood","bedrooms"], columns = ["room_type"])
price3.head(20)

price/br                         
room_type             entire home/apt private room shared room
neighborhood bedrooms                                         
ajuda        0.0                  inf          NaN         NaN
             1.0            51.041667    30.523810   18.000000
             2.0            34.825000          NaN         NaN
             3.0            32.000000          NaN         NaN
             4.0            86.250000          NaN         NaN
             9.0            25.444444          NaN         NaN
alcântara    0.0                  inf          NaN         NaN
             1.0            59.704918    42.970588         NaN
             2.0            44.818182          NaN         NaN
             3.0            28.777778          NaN         NaN
             4.0            33.416667          NaN         NaN
             6.0            40.000000          NaN         NaN
alvalade     0.0                  inf          inf         NaN
             1.0            62.218750    34.643836   29.500000
             2.0            36.775862    22.000000         NaN
             3.0            37.100000          NaN         NaN
             4.0            21.083333          NaN         NaN
             5.0            49.900000          NaN         NaN
areeiro      0.0                  NaN          inf         NaN
             1.0            70.325000    32.312500   20.642857